# 1. Create Handwritten Fakes

In [ ]:
from types import SimpleNamespace
from collections import defaultdict

class FakeQdrantClient:
    """
    • Records every call so we can assert on it.
    • Returns the smallest object that production code needs.
    """

    def __init__(self):
        self.calls = defaultdict(list)
        self.collections = {}  # Track collections for more realistic testing

    def query_points(self, **kw):
        self.calls["query_points"].append(kw)
        ScoredPoint = SimpleNamespace  # poor man's scored point
        QueryResponse = SimpleNamespace
        num_points = 1
        if "limit" in kw:  # top_k
            num_points = kw["limit"]
        points = []
        for i in range(num_points):
            p = ScoredPoint(
                id=f"P{i+1}",
                score=0.42,
                payload={"data": "hello", "metadata": {"lang": "en"}},
            )
            points.append(p)
        return QueryResponse(points=points)

    def upsert(self, **kw):
        self.calls["upsert"].append(kw)
        
        collection_name = kw.get("collection_name")
        points = kw.get("points", [])
        
        # Track points in collections for more realistic testing
        if collection_name not in self.collections:
            self.collections[collection_name] = []
        
        self.collections[collection_name].extend(points)

    def create_collection(self, **kw):
        self.calls["create_collection"].append(kw)
        collection_name = kw.get("collection_name")
        vectors_config = kw.get("vectors_config")
        sparse_vectors_config = kw.get("sparse_vectors_config")
        
        # Track created collections
        self.collections[collection_name] = []
    
    def get_collections(self):
        """Return list of collections"""
        self.calls["get_collections"].append({})
        collections = [SimpleNamespace(name=name) for name in self.collections.keys()]
        return SimpleNamespace(collections=collections)

    def get_collection(self, collection_name):
        # bare minimum “empty” object to which you can attach arbitrary attributes
        VectorParams = SimpleNamespace
        vec_cfg = {  # default: 3-dim dense vector, 0-based
            "dense": VectorParams(size=3),
            "sparse": VectorParams(size=0),
            "multi": VectorParams(size=3),
        }
        cfg = SimpleNamespace(
            params=SimpleNamespace(
                vectors=vec_cfg, sparse_vectors={"sparse": SimpleNamespace()}
            )
        )
        return SimpleNamespace(config=cfg)

The `FakeQdrantClient` class is an excellent example of a test double (specifically a spy/mock) designed to make unit testing of Qdrant-dependent code much easier. Let me explain its implementation and the purpose of the `.calls` attribute:

#### FakeQdrantClient Explained
The FakeQdrantClient class simulates the behavior of a real QdrantClient but in a controlled, predictable manner that's ideal for testing. It has several key features:

1. The `.calls` Attribute

    The `.calls` attribute is a `defaultdict(list)` that records every method call made to the fake client. This is a common pattern in test spies and is used for two main purposes:

    - **Verifying Method Calls**: It allows test code to verify that the correct methods were called with the expected parameters. For example, you can check if `query_points` was called with the right collection name and query parameters.

    - **Debugging and Inspection**: It enables you to see the sequence and details of all calls made during a test, which is invaluable for understanding how your code interacts with the Qdrant client.

    Here's how it works:
    ```python
    def query_points(self, **kw):
        self.calls["query_points"].append(kw)  # Record the call and its parameters
        # ... then return a simulated response
    ```

    In your tests, you can then assert that specific calls were made:

2. **Simulated Responses**
    For each method of the real `QdrantClient` that your code uses, `FakeQdrantClient` implements a simplified version that:

    - Records the method call in `.calls`
    - Returns a minimalistic object that mimics the structure of the real response
    
    For example, `query_points` returns a response containing the requested number of scored points, with predictable IDs, scores, and payloads.

3. **Collection Tracking**

    The class also maintains a `collections` dictionary to track created collections and their points, making the fake client's behavior more realistic across multiple operations:
    ```python
    def upsert(self, **kw):
        self.calls["upsert"].append(kw)
        collection_name = kw.get("collection_name")
        points = kw.get("points", [])
        
        # Track points in collections for more realistic testing
        if collection_name not in self.collections:
            self.collections[collection_name] = []
        
        self.collections[collection_name].extend(points)
    ```

4. **SimpleNamespace Usage**
    The implementation uses `SimpleNamespace` from the `types` module as a convenient way to create simple objects with arbitrary attributes. This is perfect for mocking complex nested response objects without having to implement their full functionality:
    ```python
    return SimpleNamespace(config=cfg)
    ```

#### Benefits of This Approach
1. **Isolation**: Tests using `FakeQdrantClient` don't require a real Qdrant server, making them faster and more reliable.

2. **Deterministic Behavior**: The fake client always returns the same predictable responses, eliminating test flakiness.

3. **Verification Capability**: The `.calls` attribute lets you verify that your code is interacting with Qdrant in the expected way.

4. **No Side Effects**: Tests don't modify a real database, so they can be run safely in any environment.

5. **Simplified Debugging**: You can easily inspect what calls were made to diagnose test failures.

This is a sophisticated and well-designed testing approach that follows best practices for unit testing code with external dependencies. The `.calls` attribute is central to making this approach effective, as it transforms the fake client from a simple stub (that just returns canned responses) into a spy (that also records how it was used).

# 2. Mocking Libraries (unittest.mock, pytest-mock)
```python
@patch('qdrant_client.QdrantClient')
def test_with_mock(mock_qdrant):
    mock_qdrant.query_points.return_value = MagicMock(
        points=[MagicMock(id="P1", score=0.42, payload={"data": "hello"})]
    )
    # Test code...
```

Pros:
- Minimal setup: No need to create a separate fake class
- Dynamic configuration: Can change behavior per test
- Complete API coverage: Automatically creates mocks for all methods
- Built-in verification: Has methods like assert_called_with()
Cons:

- "Magic": Behavior can be hard to understand and debug
- Brittle: Easy to mock wrong things or in wrong ways
- Less explicit: Behavior defined across the test rather than centralized
- Scattered configuration: Setup spread across multiple test functions
- Weak typing: No IDE autocompletion for mocked objects

# 3. Pytest Fixtures with Dependency Injection (using mock)
```python
@pytest.fixture
def qdrant_client():
    client = MagicMock()
    client.query_points.return_value = MagicMock(
        points=[MagicMock(id="P1", score=0.42, payload={"data": "hello"})]
    )
    return client

def test_with_fixture(qdrant_client):
    # Test code using qdrant_client...
```
Pros:
- Centralized configuration: Define mocked behavior once
- Reusability: Use the same fixture across tests
- Test-specific customization: Can modify fixture in each test
- Clean tests: Dependency injection keeps tests readable

Cons:

- Still using general-purpose mocks: Lacks domain specificity
- Complex fixtures can be hard to understand: Especially with parametrization
- Context switching: Need to look at fixture definition to understand test

In [ ]:
import pytest
from unittest.mock import MagicMock, patch
from types import SimpleNamespace
from qdrant_client import QdrantClient

# Example 1: Using unittest.mock directly
def test_with_direct_mock():
    with patch('qdrant_client.QdrantClient') as mock_client:
        # Configure the mock
        scored_point = SimpleNamespace(
            id="P1", 
            score=0.95, 
            payload={"content": "test document", "metadata": {"source": "test"}}
        )
        mock_client.query_points.return_value = SimpleNamespace(points=[scored_point])
        
        # Use the mock in your code
        client = QdrantClient()
        result = client.query_points(collection_name="my_collection", query_vector=[0.1, 0.2, 0.3])
        
        # Assert that the method was called with expected arguments
        mock_client.query_points.assert_called_once()
        assert result.points[0].id == "P1"
        assert result.points[0].score == 0.95

# Example 2: Using pytest fixture with dependency injection
@pytest.fixture
def mock_qdrant_client():
    client = MagicMock()
    
    # Configure query_points behavior
    scored_point = SimpleNamespace(
        id="doc123", 
        score=0.87, 
        payload={"text": "sample text", "metadata": {"lang": "en"}}
    )
    client.query_points.return_value = SimpleNamespace(points=[scored_point])
    
    # Configure get_collections behavior
    collection = SimpleNamespace(name="test_collection")
    client.get_collections.return_value = SimpleNamespace(collections=[collection])
    
    return client

def test_search_documents(mock_qdrant_client):
    # Example function that uses QdrantClient
    def search_documents(client, query_vector, collection_name, limit=5):
        response = client.query_points(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=limit
        )
        return [point.payload["text"] for point in response.points]
    
    # Test the function with our mock
    results = search_documents(
        mock_qdrant_client, 
        query_vector=[0.5, 0.2, 0.1], 
        collection_name="test_collection"
    )
    
    # Assert on results
    assert len(results) == 1
    assert results[0] == "sample text"
    
    # Verify the mock was called correctly
    mock_qdrant_client.query_points.assert_called_once_with(
        collection_name="test_collection",
        query_vector=[0.5, 0.2, 0.1],
        limit=5
    )

# Example 3: Using the custom FakeQdrantClient from the notebook
def test_with_fake_client():
    client = FakeQdrantClient()
    
    # Use the fake client in your function
    def upsert_documents(client, collection_name, documents):
        points = [{"id": f"doc{i}", "vector": [0.1, 0.2, 0.3], "payload": doc} 
                 for i, doc in enumerate(documents)]
        client.upsert(collection_name=collection_name, points=points)
        return len(points)
    
    # Call the function
    docs = [{"text": "hello"}, {"text": "world"}]
    count = upsert_documents(client, "my_docs", docs)
    
    # Assert that the right calls were made
    assert len(client.calls["upsert"]) == 1
    assert client.calls["upsert"][0]["collection_name"] == "my_docs"
    assert len(client.calls["upsert"][0]["points"]) == 2
    assert count == 2
    
    # Verify the documents were stored in the fake client's collections
    assert len(client.collections["my_docs"]) == 2
    assert client.collections["my_docs"][0]["payload"]["text"] == "hello"